In [ ]:
!pip install -q roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="P1tOolfxuLaCFYy4PyEb")
project = rf.workspace("naspadsupremasi").project("citrus-disease-cjcbq")
version = project.version(7)
dataset = version.download("coco-segmentation")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.3 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to citrus-disease-7 in coco-segmentation:: 100%|██████████| 2004/2004 [00:00<00:00, 3916.41it/s]


In [ ]:
!mkdir /content/citrus-disease-7/mask

In [ ]:
import json
import numpy as np
import cv2
from pycocotools import mask as maskUtils
from tqdm import tqdm
import os

coco_json_path = '/content/citrus-disease-7/train/_annotations.coco.json'
img_dir = '/content/citrus-disease-7/train'
mask_output_dir = '/content/citrus-disease-7/mask'


with open(coco_json_path) as f:
    coco = json.load(f)

# Map image_id to file_name
id_to_filename = {img['id']: img['file_name'] for img in coco['images']}

# Group annotations by image_id
from collections import defaultdict
ann_dict = defaultdict(list)
for ann in coco['annotations']:
    ann_dict[ann['image_id']].append(ann)

cat_id_to_class = {
    0: 1,  # lesion
    1: 2,  # fruit
    2: 1   # lesion
}

class_priority = {
    0: 2,  # lesion
    1: 1,  # fruit
    2: 2   # lesion
}

for img_id, anns in tqdm(ann_dict.items()):
    img_info = next(item for item in coco['images'] if item['id'] == img_id)
    height, width = img_info['height'], img_info['width']
    mask = np.zeros((height, width), dtype=np.uint8)

    # Sort anns by priority (fruit first, lesion last)
    anns = sorted(anns, key=lambda x: class_priority.get(x['category_id'], 0))

    for ann in anns:
        segmentation = ann['segmentation']
        category_id = ann['category_id']
        class_id = cat_id_to_class.get(category_id, 0)

        if isinstance(segmentation, list):
            for seg in segmentation:
                pts = np.array(seg).reshape(-1, 2).astype(np.int32)
                if pts.shape[0] >= 3:
                    cv2.fillPoly(mask, [pts], color=class_id)
        else:
            rle = maskUtils.frPyObjects(segmentation, height, width)
            m = maskUtils.decode(rle)
            mask[m > 0] = class_id

    filename = id_to_filename[img_id].replace('.jpg', '.png')
    cv2.imwrite(os.path.join(mask_output_dir, filename), mask * 100)



100%|██████████| 2000/2000 [00:10<00:00, 191.41it/s]


In [ ]:
!rm -r /content/citrus-disease-7/train/_annotations.coco.json

In [ ]:
import os
from glob import glob
from os.path import basename
import cv2

image_paths = sorted(glob('/content/citrus-disease-7/train/*.jpg'))

# Build mask paths: use actual if exists, else create an empty placeholder later
mask_dir = '/content/citrus-disease-7/mask'
mask_paths = []

for img_path in image_paths:
    name = basename(img_path).replace('.jpg', '')
    mask_path = os.path.join(mask_dir, f'{name}.png')

    if os.path.exists(mask_path):
        mask_paths.append(mask_path)
    else:
        # Mark empty with a placeholder (we’ll generate in dataset class)
        mask_paths.append(None)


In [ ]:
!pip install -q segmentation-models-pytorch albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.6 MB/s eta 0:00:00


In [ ]:
import os
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2


In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.images = sorted(os.listdir(images_dir))
        self.masks = sorted(os.listdir(masks_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = np.array(Image.open(os.path.join(self.images_dir, self.images[idx])).convert("RGB"))
        mask = np.array(Image.open(os.path.join(self.masks_dir, self.masks[idx])))

        # Map unique pixel values to class indices
        mask = np.where(mask == 100, 1, mask)
        mask = np.where(mask == 200, 2, mask)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask'].long()

        return image, mask


In [ ]:
train_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])
from torch.utils.data import random_split

# Full dataset
full_dataset = SegmentationDataset(
    images_dir='/content/citrus-disease-7/train',
    masks_dir='/content/citrus-disease-7/mask',
    transform=train_transform
)

# Split sizes
val_percent = 0.2
val_size = int(val_percent * len(full_dataset))
train_size = len(full_dataset) - val_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


In [ ]:
model = smp.Unet(
    encoder_name="vgg16",
    encoder_weights="imagenet",
    in_channels=3,
    classes=3,  # 3 classes (0, 1, 2)
)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

Unet(
  (encoder): VGGEncoder(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)


In [ ]:
def compute_iou(preds, labels, num_classes):
    ious = []
    preds = preds.view(-1)
    labels = labels.view(-1)

    for cls in range(num_classes):
        pred_inds = preds == cls
        label_inds = labels == cls
        intersection = (pred_inds & label_inds).sum().item()
        union = (pred_inds | label_inds).sum().item()
        if union == 0:
            ious.append(float('nan'))  # Ignore class if not present
        else:
            ious.append(intersection / union)

    return ious
def evaluate_iou(model, dataloader, num_classes, device):
    model.eval()
    iou_list = []

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            ious = compute_iou(preds, masks, num_classes)
            iou_list.append(ious)

    # Average over all batches
    iou_array = np.array(iou_list)
    mean_iou = np.nanmean(iou_array, axis=0)  # per-class IoU
    miou = np.nanmean(mean_iou)              # mean IoU across classes
    return mean_iou, miou


In [ ]:

for epoch in range(25):
    model.train()
    total_loss = 0
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device)
            masks = masks.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            val_loss += loss.item()

    mean_iou, miou = evaluate_iou(model, val_loader, num_classes=3, device=device)
    print(f"Epoch {epoch+1}: Val Loss = {val_loss / len(val_loader):.4f}, mIoU = {miou:.4f}, Class IoUs = {mean_iou}")


Epoch 1: Val Loss = 0.1060, mIoU = 0.6750, Class IoUs = [0.98411044 0.08502754 0.95595681]
Epoch 2: Val Loss = 0.0590, mIoU = 0.8646, Class IoUs = [0.98740046 0.63038702 0.97615125]
Epoch 3: Val Loss = 0.0570, mIoU = 0.8265, Class IoUs = [0.98723143 0.5191418  0.97316138]
Epoch 4: Val Loss = 0.0489, mIoU = 0.8621, Class IoUs = [0.98737707 0.62368995 0.97536931]
Epoch 5: Val Loss = 0.0351, mIoU = 0.8984, Class IoUs = [0.98946456 0.72392685 0.9817031 ]
Epoch 6: Val Loss = 0.0348, mIoU = 0.8875, Class IoUs = [0.98848369 0.69369096 0.98024103]
Epoch 7: Val Loss = 0.0302, mIoU = 0.9058, Class IoUs = [0.98904738 0.74628056 0.98213339]
Epoch 8: Val Loss = 0.0272, mIoU = 0.9129, Class IoUs = [0.98996288 0.76449349 0.98423565]
Epoch 9: Val Loss = 0.0307, mIoU = 0.9038, Class IoUs = [0.98961801 0.74101389 0.98062195]
Epoch 10: Val Loss = 0.0244, mIoU = 0.9203, Class IoUs = [0.99018998 0.78564958 0.98504749]
Epoch 11: Val Loss = 0.0235, mIoU = 0.9232, Class IoUs = [0.9903106  0.7938937  0.9854758

In [ ]:
!pip install torchinfo -q
from torchinfo import summary

summary(model, input_size=(1, 3, 256, 256))  # batch size 1


Layer (type:depth-idx)                        Output Shape              Param #
Unet                                          [1, 3, 256, 256]          --
├─VGGEncoder: 1-1                             [1, 64, 256, 256]         --
│    └─Sequential: 2-1                        --                        --
│    │    └─Conv2d: 3-1                       [1, 64, 256, 256]         1,792
│    │    └─ReLU: 3-2                         [1, 64, 256, 256]         --
│    │    └─Conv2d: 3-3                       [1, 64, 256, 256]         36,928
│    │    └─ReLU: 3-4                         [1, 64, 256, 256]         --
│    │    └─MaxPool2d: 3-5                    [1, 64, 128, 128]         --
│    │    └─Conv2d: 3-6                       [1, 128, 128, 128]        73,856
│    │    └─ReLU: 3-7                         [1, 128, 128, 128]        --
│    │    └─Conv2d: 3-8                       [1, 128, 128, 128]        147,584
│    │    └─ReLU: 3-9                         [1, 128, 128, 128]        --
│   